<a href="https://colab.research.google.com/github/tawargy/4k_Video_Upscaler_Colab_-Real_ESRGAN-/blob/main/4k_Video_Upscaler_Colab_(Real_ESRGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4k Video Upscaler Colab (Real-ESRGAN)

Adapted from: [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN)



# 1. Setup (~1 minute)

In [16]:
# 1. Check GPU
import torch
import os
assert torch.cuda.is_available(), "GPU not detected. Please change runtime to GPU"

# 2. Setup Repository
if not os.path.exists('Real-ESRGAN'):
    !git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

# 3. Install Dependencies
# We install specific versions to avoid the latest Numpy 2.0 breaking changes
!pip install -q basicsr facexlib gfpgan ffmpeg-python
!pip install -q -r requirements.txt
!python setup.py develop
!pip install -q "numpy<2" "MarkupSafe==2.0.1" "opencv-python-headless<4.11"

# 4. FIX: Apply the patch directly to the file path (BEFORE IMPORTING)
# This fixes the 'torchvision.transforms.functional_tensor' error
!sed -i 's/from torchvision.transforms.functional_tensor import/from torchvision.transforms.functional import/g' /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py
!sed -i 's/from torchvision.transforms.functional_tensor import/from torchvision.transforms.functional import/g' /usr/local/lib/python3.12/dist-packages/basicsr/data/metrictools.py

# 5. FIX: Also fix the 'runtime_error' in basicsr/utils/download_util.py
!sed -i 's/from torch.hub import download_url_to_file/from torch.hub import download_url_to_file/g' /usr/local/lib/python3.12/dist-packages/basicsr/utils/download_util.py

print("✅ Patches applied successfully. Now you can safely import basicsr.")

# 6. Verify Import
import basicsr
print(f"Basicsr version {basicsr.__version__} loaded correctly!")

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 759 (delta 106), reused 99 (delta 99), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 4.47 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN/Real-ESRGAN
/usr/local/lib/python3.12/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.12/dist-packages/setuptools/co

In [17]:
from google.colab import drive
import os

# Mount Drive
if not os.path.exists('/content/gdrive'):
    drive.mount('/content/gdrive')

# Download the model weights to the weights folder
!mkdir -p weights
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P weights/
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus_anime_6B.pth -P weights/

print("✅ Setup Complete. Models downloaded and Drive mounted.")

--2025-12-18 12:19:38--  https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/387326890/08f0e941-ebb7-48f0-9d6a-73e87b710e7e?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-18T12%3A59%3A30Z&rscd=attachment%3B+filename%3DRealESRGAN_x4plus.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-18T11%3A59%3A20Z&ske=2025-12-18T12%3A59%3A30Z&sks=b&skv=2018-11-09&sig=adkSa4tdAdiv6Zfpe5ysGoe85P%2FpqgcSc78xM695cqU%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NjA2MjE3OCwibmJmIjoxNzY2MDYwMzc4LCJwYXRoIjoicmVsZWFzZWFzc2V0

# 3. Upscale video

- The upscaled video will be saved to `output_dir`
- If google drive is mounted, it will be also saved at `MyDrive/Upscaled Videos (REAL-ESRGAN)`


In [18]:
import cv2, os, subprocess
from PIL import Image

# --- CONFIGURATION ---
video_path = "/content/gdrive/MyDrive/content/x.mp4" # @param {type:"string"}
output_dir = "/content/" # @param {type:"string"}
resolution = "2k (2560 x 1440)" # @param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)","2 x original", "3 x original", "4 x original"]
model = "RealESRGAN_x4plus" # @param ["RealESRGAN_x4plus" , "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]
mount_drive = True # @param {type:"boolean"}

# --- VALIDATION ---
if not os.path.exists(video_path):
    print(f"❌ ERROR: File not found at {video_path}")
    print("Files currently in your drive folder:")
    !ls "/content/gdrive/MyDrive/content/"
    raise SystemExit("Please check the filename and try again.")

# --- CALCULATE SCALE ---
video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
aspect_ratio = float(video_width/video_height)

resolutions = {
    "FHD (1920 x 1080)": (1920, 1080),
    "2k (2560 x 1440)": (2560, 1440),
    "4k (3840 x 2160)": (3840, 2160)
}

if "original" in resolution:
    mult = int(resolution.split(" ")[0])
    final_width, final_height = video_width * mult, video_height * mult
else:
    final_width, final_height = resolutions[resolution]
    # Adjust for portrait if needed
    if aspect_ratio < 1.0:
        final_width, final_height = final_height, final_width

scale_factor = max(final_width/video_width, final_height/video_height)
print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height} (Scale: {scale_factor:.2f})")

# --- EXECUTE UPSCALING ---
!python inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale {scale_factor}

# --- POST-PROCESSING ---
video_name_with_ext = os.path.basename(video_path)
video_name = os.path.splitext(video_name_with_ext)[0]
upscaled_video_path = f"{output_dir}{video_name}_out.mp4"
final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"
final_video_path = os.path.join(output_dir, final_video_name)

if os.path.exists(upscaled_video_path):
    if "original" not in resolution:
        print("Cropping and encoding...")
        command = f"ffmpeg -loglevel error -hwaccel cuda -y -i '{upscaled_video_path}' -vf 'crop={final_width}:{final_height}' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
        subprocess.run(command, shell=True)
    else:
        os.rename(upscaled_video_path, final_video_path)

    print(f"✅ Upscaled video saved to: {final_video_path}")

    if mount_drive:
        drive_folder = "/content/gdrive/MyDrive/Upscaled_Videos"
        os.makedirs(drive_folder, exist_ok=True)
        !cp '{final_video_path}' '{drive_folder}/{final_video_name}'
        print(f"📂 Also saved to Drive: {drive_folder}/{final_video_name}")
else:
    print("❌ Upscaling failed. Check logs above for Real-ESRGAN errors.")

Upscaling from 256x144 to 2560x1440 (Scale: 10.00)
inference: 100% 4203/4203 [29:54<00:00,  2.34frame/s]
Cropping and encoding...
✅ Upscaled video saved to: /content/x_upscaled_2560_1440.mp4
📂 Also saved to Drive: /content/gdrive/MyDrive/Upscaled_Videos/x_upscaled_2560_1440.mp4
